# 🧬 Cortex-2: Advanced Evolutionary Research Lab

## 📜 Estado del Arte y Justificación Científica
Este entorno implementa una arquitectura de frontera basada en los siguientes papers disruptivos:

1.  **Mamba (SSM)**: *Gu, A., & Dao, T. (2023). "Mamba: Linear-Time Sequence Modeling with Selective State Spaces".* [arXiv:2312.00752](https://arxiv.org/abs/2312.00752)
    *   *Por qué*: Resuelve el cuello de botella cuadrático de los Transformers ($O(N^2)$) permitiendo contextos infinitos con coste lineal ($O(N)$).
2.  **Mixture of Experts (MoE)**: *Shazeer et al. (2017). "Outrageously Large Neural Networks: The Sparsely-Gated Mixture-of-Experts Layer".* [arXiv:1701.06538](https://arxiv.org/abs/1701.06538)
    *   *Por qué*: Desacopla la capacidad de computación (FLOPs) de la capacidad de memoria (Parámetros). Permite modelos gigantes que corren rápido.
3.  **Byte-Level Modeling**: *Xue et al. (2022). "ByT5: Towards a Token-Free Future".* [arXiv:2105.13626](https://arxiv.org/abs/2105.13626)
    *   *Por qué*: Elimina el sesgo humano del Tokenizer. Hace al modelo robusto a "ruido" y multilingüe por defecto.
4.  **Neural Architecture Search (NAS)**: *Real et al. (2019). "Regularized Evolution for Image Classifier Architecture Search".* [arXiv:1802.01548](https://arxiv.org/abs/1802.01548)
    *   *Por qué*: La intuición humana falla en espacios de alta dimensión. La evolución encuentra óptimos locales que nosotros ignoramos.

---

In [ ]:
# 0. Configuración e Importaciones
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA
from IPython.display import clear_output, display

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Cortex-2 Engine Active on: {device.upper()}")

## 🧠 Arquitectura Modular (Caja Blanca)
Hemos instrumentado el código para extraer **telemetría interna**. No es solo "forward pass", es un escáner cerebral.

In [ ]:
# --- Componentes Instrumentados ---

class InstrumentedAttention(nn.Module):
    """ Atención con captura de mapas de calor """
    def __init__(self, config):
        super().__init__()
        self.mha = nn.MultiheadAttention(config['d_model'], config['n_heads'], batch_first=True)
        self.last_attn_weights = None
        
    def forward(self, x):
        # Capturamos los pesos de atención (Average across heads for simplicity in visualization)
        out, weights = self.mha(x, x, x, need_weights=True, average_attn_weights=True)
        self.last_attn_weights = weights.detach().cpu()
        return out

class MambaBlock(nn.Module):
    """ Bloque Mamba Simplificado """
    def __init__(self, d_model):
        super().__init__()
        self.in_proj = nn.Linear(d_model, d_model * 2)
        self.out_proj = nn.Linear(d_model, d_model)
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, groups=d_model)
        
    def forward(self, x):
        B, L, D = x.shape
        x_and_res = self.in_proj(x)
        x_val, res = x_and_res.chunk(2, dim=-1)
        x_val = x_val.transpose(1, 2)
        x_val = self.conv(x_val)
        x_val = x_val.transpose(1, 2)
        x_val = F.silu(x_val)
        return self.out_proj(x_val * F.sigmoid(res))

class MoELayer(nn.Module):
    """ MoE con Telemetría de Routing """
    def __init__(self, d_model, n_experts, top_k=2):
        super().__init__()
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(d_model, 4 * d_model), nn.GELU(), 
                nn.Linear(4 * d_model, d_model), nn.Dropout(0.1)
            ) for _ in range(n_experts)
        ])
        self.gate = nn.Linear(d_model, n_experts)
        self.top_k = top_k
        self.last_routing_dist = None

    def forward(self, x):
        gate_logits = self.gate(x)
        weights, indices = torch.topk(gate_logits, self.top_k, dim=-1)
        weights = F.softmax(weights, dim=-1)
        
        # Telemetría: ¿Qué expertos se activaron?
        self.last_routing_dist = indices.detach().cpu().view(-1).bincount(minlength=len(self.experts))
        
        out = torch.zeros_like(x)
        for i, expert in enumerate(self.experts):
            mask = (indices == i).any(dim=-1)
            if mask.any():
                out[mask] += expert(x[mask])
        return out

class CortexOrganism(nn.Module):
    def __init__(self, dna):
        super().__init__()
        self.dna = dna
        self.embedding = nn.Embedding(256, dna['d_model'])
        self.layers = nn.ModuleList()
        
        for i in range(dna['n_layers']):
            if dna['backbone'] == 'mamba':
                self.layers.append(MambaBlock(dna['d_model']))
            elif dna['backbone'] == 'hybrid' and i % 2 == 0:
                self.layers.append(MambaBlock(dna['d_model']))
            else:
                # Usamos nuestra Atención Instrumentada
                self.layers.append(InstrumentedAttention(dna))
        
        if dna['moe_experts'] > 0:
            self.final_layer = MoELayer(dna['d_model'], dna['moe_experts'])
        else:
            self.final_layer = nn.Linear(dna['d_model'], dna['d_model'])
            
        self.ln_f = nn.LayerNorm(dna['d_model'])
        self.head = nn.Linear(dna['d_model'], 256)

    def forward(self, idx, targets=None):
        x = self.embedding(idx)
        for layer in self.layers:
            x = layer(x)
        if self.dna['moe_experts'] > 0:
            x = self.final_layer(x)
        x = self.ln_f(x)
        logits = self.head(x)
        
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

## 🔬 Visualización Avanzada
Aquí definimos las herramientas para inspeccionar el modelo.

In [ ]:
def visualize_attention(model, input_bytes):
    """ Muestra qué bytes miran a qué bytes """
    # Buscar la última capa de atención
    attn_layer = None
    for layer in model.layers:
        if isinstance(layer, InstrumentedAttention):
            attn_layer = layer
            
    if attn_layer is None or attn_layer.last_attn_weights is None:
        print("⚠️ No hay capas de atención activas o registradas.")
        return

    weights = attn_layer.last_attn_weights[0] # Primer batch
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(weights.numpy(), cmap='viridis')
    plt.title("Mapa de Calor de Atención (Razonamiento)")
    plt.xlabel("Key Token")
    plt.ylabel("Query Token")
    plt.show()

def visualize_moe_routing(model):
    """ Muestra la carga de trabajo de cada experto """
    if not hasattr(model, 'final_layer') or not isinstance(model.final_layer, MoELayer):
        return
        
    dist = model.final_layer.last_routing_dist
    if dist is None: return
    
    plt.figure(figsize=(8, 4))
    plt.bar(range(len(dist)), dist.numpy(), color='#818cf8')
    plt.title("Distribución de Carga de Expertos (MoE)")
    plt.xlabel("ID del Experto")
    plt.ylabel("Tokens Procesados")
    plt.show()

## 🔮 Futuro y Alternativas

**¿Qué falta? ¿Dónde indagar más?**

1.  **RWKV (Receptance Weighted Key Value)**: *Peng et al.*. Una alternativa fascinante que entrena como Transformer (paralelo) pero infiere como RNN (serial). Deberíamos probar añadir bloques RWKV al genoma.
2.  **Griffin / Hawk**: *Google DeepMind*. Arquitecturas híbridas Recurrentes-Atencionales más recientes que Mamba.
3.  **Optimizadores Exóticos**: Estamos usando AdamW. Deberíamos probar **Sophia** (Second-order Clipped Optimization) o **Muon** (Momentum Orthogonalizer) para convergencia más rápida.
4.  **Curriculum Learning**: El scraper descarga papers, pero deberíamos ordenarlos por complejidad (Lógica -> Ciencia -> Literatura) automáticamente.

In [ ]:
# --- Genoma y Bucle Evolutivo (Igual que antes, pero usando las nuevas clases) ---
class Genome:
    def __init__(self):
        self.genes = {
            'n_layers': [2, 4, 6], 
            'd_model': [128, 256], 
            'n_heads': [2, 4], 
            'backbone': ['transformer', 'hybrid'], 
            'moe_experts': [0, 4, 8], 
            'learning_rate': [1e-3, 3e-4]
        }
        self.dna = {k: random.choice(v) for k, v in self.genes.items()}
        self.fitness = 0.0
    
    def mutate(self):
        k = random.choice(list(self.genes.keys()))
        self.dna[k] = random.choice(self.genes[k])
        return f"🧬 Mutación: {k} -> {self.dna[k]}"
        
    def crossover(self, other):
        child = Genome()
        for k in self.genes:
            child.dna[k] = self.dna[k] if random.random() > 0.5 else other.dna[k]
        return child

# Datos Dummy
dummy_data = torch.randint(0, 256, (1000,), dtype=torch.long)
def get_batch():
    ix = torch.randint(len(dummy_data) - 32, (16,))
    x = torch.stack([dummy_data[i:i+32] for i in ix]).to(device)
    y = torch.stack([dummy_data[i+1:i+33] for i in ix]).to(device)
    return x, y

def run_evolution(generations=3, pop_size=4):
    population = [Genome() for _ in range(pop_size)]
    
    for gen in range(generations):
        print(f"\n🏁 Generación {gen+1}")
        for i, genome in enumerate(population):
            model = CortexOrganism(genome.dna).to(device)
            optim = torch.optim.AdamW(model.parameters(), lr=genome.dna['learning_rate'])
            
            # Sprint Training
            losses = []
            for _ in range(10):
                xb, yb = get_batch()
                _, loss = model(xb, yb)
                optim.zero_grad()
                loss.backward()
                optim.step()
                losses.append(loss.item())
            
            genome.fitness = sum(losses[-3:]) / 3
            print(f"   Individuo {i} ({genome.dna['backbone']}): Loss {genome.fitness:.4f}")
            
            # Visualizar al campeón de la ronda
            if i == 0:
                visualize_attention(model, xb)
                visualize_moe_routing(model)
        
        # Selección
        population.sort(key=lambda x: x.fitness)
        survivors = population[:pop_size//2]
        
        # Reproducción
        new_pop = survivors[:]
        while len(new_pop) < pop_size:
            parent = random.choice(survivors)
            child = parent.crossover(random.choice(survivors))
            if random.random() < 0.4: child.mutate()
            new_pop.append(child)
        population = new_pop

run_evolution()